In [7]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import tkinter as tk
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os

nltk.download('stopwords')

# defining suspicious scam words
scam_words = "aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery"
scam_words = scam_words.split(" ")
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.".lower(), "True"],
    ["Please provide me with your Aadhaar Card details.".lower(), "True"],
    ["Can you tell us your Credit Card CVV number.".lower(), "True"],
    ["Please provide me with your Credit Card number.".lower(), "True"],
    ["What is your address.".lower(), "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.".lower(), "True"],
    ["I am your father's friend, he told me to ask you for some money.".lower(), "True"],
    ["Kindly tell me your Username and Password of your Google account.".lower(), "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.".lower(), "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.".lower(), "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.".lower(), "False"],
    ["Kindly go to our official website and provide us with the necessary details.".lower(), "False"],
    ["Visit the nearest branch of XYZ bank to process your request.".lower(), "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.".lower(), "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.".lower(), "False"],
    ["Tell your mom that XYZ aunty had called.".lower(), "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.".lower(), "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?".lower(), "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.".lower(), "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm".lower(), "False"]
]

def get_large_audio_transcription(path, pitch=0):
    """
    Split the large audio file into chunks, apply pitch shifting, and perform speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    # Pitch shifting
    if pitch != 0:
       
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': int(sound.frame_rate * (2.0 ** (pitch / 12.0)))})

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text


def filter_common_words(text_list):
    common_words = stopwords.words("english")
    # common words like ["me","your","to","a"] which have no contribution to detection
    word_seq = [i for i in text_list if (i not in common_words)]
    return word_seq


def scam_probability(sentence):  # Calculates Scam probability of single Sentence(List)
    sentence = filter_common_words(sentence)
    sentence = [word.lower() for word in sentence]
    if set(sentence).difference(scam_words) == set(sentence):
        return 0
    else:
        scam_indices = [i for i in range(len(sentence)) if sentence[i] in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(sentence)
        elif len(scam_indices) / len(sentence) > 0.5:
            return len(scam_indices) / len(sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(sentence))
    return probability


def batch_scam_probability(sentences):  # Calculates Scam probability of multiple Sentences(List of lists)
    probabilities = [scam_probability(sentence) for sentence in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)


def calculate_accuracy(path, pitch=0):
    text = get_large_audio_transcription(path, pitch)
    text_list = text.split(" ")
    prob = scam_probability(text_list)
    return prob


# Code for GUI

customtkinter.set_appearance_mode("light")  # Modes: system (default), light, dark
customtkinter.set_default_color_theme("dark-blue")  # Themes: blue (default), dark-blue, green

app = customtkinter.CTk()  # create CTk window like you do with the Tk window
app.geometry("300x500")
app.title("Scam Call Detector v1.2")


def get_path():
    path = entry.get()    
    pitch = int(pitch_entry.get())
    probability = calculate_accuracy(path, pitch)

    # Update the label text using the configure method
    result_label.configure(text="Scam Probability: {:.2%}".format(probability))



# Create a label for the entry field
entry_label = customtkinter.CTkLabel(app, text="Enter the path to the audio file:")
entry_label.pack(pady=10)

# Create an entry field for the path
entry = customtkinter.CTkEntry(app, width=250)
entry.pack()

# Create a label for the pitch entry field
pitch_label = customtkinter.CTkLabel(app, text="Enter the pitch value:")
pitch_label.pack(pady=10)

# Create an entry field for the pitch
pitch_entry = customtkinter.CTkEntry(app, width=30)
pitch_entry.pack()

# Create a button to trigger the analysis
analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
analyze_button.pack(pady=10)

# Create a label to display the result
result_label = customtkinter.CTkLabel(app, text="")
result_label.pack(pady=10)

app.mainloop()  # run the app loop

   


[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: ['aadhaar', 'cvv', 'username', 'password', 'access', 'information', 'special', 'lucky', 'money', 'OTP', 'credit', 'card', 'bonus', 'code', 'chance', 'promo', 'win', 'last', 'contact', 'prizes', 'rewards', 'phone', 'send', 'give', 'forward', 'email', 'message', 'account', 'number', 'lottery']


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\site-packages\speech_recognition\__init__.py", line 894, in recognize_google
    response = urlopen(request, timeout=self.operation_timeout)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 649, in h

KeyboardInterrupt: 

In [4]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import tkinter as tk
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os

nltk.download('stopwords')

# defining suspicious scam words
scam_words = "aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery"
scam_words = scam_words.split(" ")
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.".lower(), "True"],
    ["Please provide me with your Aadhaar Card details.".lower(), "True"],
    ["Can you tell us your Credit Card CVV number.".lower(), "True"],
    ["Please provide me with your Credit Card number.".lower(), "True"],
    ["What is your address.".lower(), "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.".lower(), "True"],
    ["I am your father's friend, he told me to ask you for some money.".lower(), "True"],
    ["Kindly tell me your Username and Password of your Google account.".lower(), "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.".lower(), "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.".lower(), "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.".lower(), "False"],
    ["Kindly go to our official website and provide us with the necessary details.".lower(), "False"],
    ["Visit the nearest branch of XYZ bank to process your request.".lower(), "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.".lower(), "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.".lower(), "False"],
    ["Tell your mom that XYZ aunty had called.".lower(), "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.".lower(), "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?".lower(), "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.".lower(), "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm".lower(), "False"]
]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    """
    Split the large audio file into chunks, apply pitch shifting and speed adjustment,
    and perform speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

       # Pitch shifting
    if pitch != 0:
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': int(sound.frame_rate * (2.0 ** (pitch / 12.0)))})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text


def filter_common_words(text_list):
    common_words = stopwords.words("english")
    # common words like ["me","your","to","a"] which have no contribution to detection
    word_seq = [i for i in text_list if i not in common_words]
    return word_seq


def scam_probability(sentence):  # Calculates Scam probability of a single Sentence (List)
    sentence = filter_common_words(sentence)
    sentence = [word.lower() for word in sentence]
    if set(sentence).difference(scam_words) == set(sentence):
        return 0
    else:
        scam_indices = [i for i in range(len(sentence)) if sentence[i] in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(sentence)
        elif len(scam_indices) / len(sentence) > 0.5:
            return len(scam_indices) / len(sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(sentence))
    return probability


def batch_scam_probability(sentences):  # Calculates Scam probability of multiple Sentences (List of lists)
    probabilities = [scam_probability(sentence) for sentence in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)


# Existing code ...
def calculate_accuracy(path, pitch=0, speed=1.0):
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text_list)
    return prob



# Code for GUI

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")  # Modes: system (default), light, dark
    customtkinter.set_default_color_theme("dark-blue")  # Themes: blue (default), dark-blue, green

    app = customtkinter.CTk()  # create CTk window like you do with the Tk window
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    def get_path():
        path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        probability = calculate_accuracy(path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))

    # Create a label for the entry field
    entry_label = customtkinter.CTkLabel(app, text="Enter the path to the audio file:")
    entry_label.pack(pady=10)

    # Create an entry field for the path
    entry = customtkinter.CTkEntry(app, width=250)
    entry.pack()

    # Create a label for the pitch entry field
    pitch_label = customtkinter.CTkLabel(app, text="Enter the pitch value:")
    pitch_label.pack(pady=10)

    # Create an entry field for the pitch
    pitch_entry = customtkinter.CTkEntry(app, width=10)
    pitch_entry.pack()

    # Create a label for the speed entry field
    speed_label = customtkinter.CTkLabel(app, text="Enter the speech rate (1.0 for normal speed):")
    speed_label.pack(pady=10)

    # Create an entry field for the speed
    speed_entry = customtkinter.CTkEntry(app, width=10)
    speed_entry.pack()

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop() 

[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: ['aadhaar', 'cvv', 'username', 'password', 'access', 'information', 'special', 'lucky', 'money', 'OTP', 'credit', 'card', 'bonus', 'code', 'chance', 'promo', 'win', 'last', 'contact', 'prizes', 'rewards', 'phone', 'send', 'give', 'forward', 'email', 'message', 'account', 'number', 'lottery']


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\site-packages\speech_recognition\__init__.py", line 894, in recognize_google
    response = urlopen(request, timeout=self.operation_timeout)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Users\Himesh Punj\.conda\envs\NEWENV\lib\urllib\request.py", line 649, in h

In [8]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import tkinter as tk
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os

# Download stopwords outside the function
nltk.download('stopwords')

# defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

# Preprocess scam and non-scam sentences
scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]


def get_large_audio_transcription(path, pitch=0, speed=1.0):
    """
    Split the large audio file into chunks, apply pitch shifting and speed adjustment,
    and perform speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    # Pitch shifting
    if pitch != 0:
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': int(sound.frame_rate * (2.0 ** (pitch / 12.0)))})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text


def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]


def scam_probability(sentence):  # Calculates Scam probability of a single Sentence (List)
    filtered_sentence = filter_common_words(sentence)
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability
def batch_scam_probability(sentences):  # Calculates Scam probability of multiple Sentences (List of lists)
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)


# Existing code ...

def calculate_accuracy(path, pitch=0, speed=1.0):
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text_list)
    return prob


# Code for GUI

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")  # Modes: system (default), light, dark
    customtkinter.set_default_color_theme("dark-blue")  # Themes: blue (default), dark-blue, green

    app = customtkinter.CTk()  # create CTk window like you do with the Tk window
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    def get_path():
        path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        probability = calculate_accuracy(path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2",font=('Century Gothic', 20))
    entry_label.pack(pady=10)
    # Create a label for the entry field
    entry_label = customtkinter.CTkLabel(app, text="Enter the path to the audio file:")
    entry_label.pack(pady=10)

    # Create an entry field for the path
    entry = customtkinter.CTkEntry(app, width=250)
    entry.pack()

    # Create a label for the pitch entry field
    pitch_label = customtkinter.CTkLabel(app, text="Enter the pitch value:")
    pitch_label.pack(pady=10)

    # Create an entry field for the pitch
    pitch_entry = customtkinter.CTkEntry(app, width=50)
    pitch_entry.pack()

    # Create a label for the speed entry field
    speed_label = customtkinter.CTkLabel(app, text="Enter the speech rate (1.0 for normal speed):")
    speed_label.pack(pady=10)

    # Create an entry field for the speed
    speed_entry = customtkinter.CTkEntry(app, width=50)
    speed_entry.pack()

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop()



[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: {'email', 'OTP', 'prizes', 'code', 'win', 'rewards', 'forward', 'account', 'number', 'send', 'credit', 'phone', 'cvv', 'special', 'lottery', 'information', 'chance', 'bonus', 'access', 'last', 'card', 'password', 'promo', 'contact', 'give', 'message', 'lucky', 'aadhaar', 'username', 'money'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Himesh Punj\.conda\envs\env1\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 553, in _clicked
    self._command()
  File "C:\Users\Himesh Punj\AppData\Local\Temp\ipykernel_1064\2972148729.py", line 128, in get_path
    probability = calculate_accuracy(path, pitch, speed)
  File "C:\Users\Himesh Punj\AppData\Local\Temp\ipykernel_1064\2972148729.py", line 109, in calculate_accuracy
    text_list = text.split(" ")
AttributeError: 'NoneType' object has no attribute 'split'


In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import tkinter as tk
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os

# Download stopwords outside the function
nltk.download('stopwords')

# defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

# Preprocess scam and non-scam sentences
scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    """
    Split the large audio file into chunks, apply pitch shifting and speed adjustment,
    and perform speech recognition on each of these chunks
    """
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    max_frame_rate = 48000  # Define the maximum frame rate here (adjust as needed)

    # Pitch shifting
    if pitch != 0:
        pitch_factor = 2.0 ** (pitch / 12.0)
        new_frame_rate = int(sound.frame_rate * pitch_factor)
        if new_frame_rate > max_frame_rate:
            pitch_factor = max_frame_rate / sound.frame_rate
            new_frame_rate = max_frame_rate
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_frame_rate})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
                text = None
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text




def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]


def scam_probability(sentence):  # Calculates Scam probability of a single Sentence (List)
    filtered_sentence = filter_common_words(sentence)
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability
def batch_scam_probability(sentences):  # Calculates Scam probability of multiple Sentences (List of lists)
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)


# Existing code ...

def calculate_accuracy(path, pitch=0, speed=1.0):
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text_list)
    return prob


# Code for GUI

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")  # Modes: system (default), light, dark
    customtkinter.set_default_color_theme("dark-blue")  # Themes: blue (default), dark-blue, green

    app = customtkinter.CTk()  # create CTk window like you do with the Tk window
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    def get_path():
        path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        probability = calculate_accuracy(path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2",font=('Century Gothic', 20))
    entry_label.pack(pady=10)
    # Create a label for the entry field
    entry_label = customtkinter.CTkLabel(app, text="Enter the path to the audio file:")
    entry_label.pack(pady=10)

    # Create an entry field for the path
    entry = customtkinter.CTkEntry(app, width=250)
    entry.pack()

    # Create a label for the pitch entry field
    pitch_label = customtkinter.CTkLabel(app, text="Enter the pitch value:")
    pitch_label.pack(pady=10)

    # Create an entry field for the pitch
    pitch_entry = customtkinter.CTkEntry(app, width=50)
    pitch_entry.pack()

    # Create a label for the speed entry field
    speed_label = customtkinter.CTkLabel(app, text="Enter the speech rate (1.0 for normal speed):")
    speed_label.pack(pady=10)

    # Create an entry field for the speed
    speed_entry = customtkinter.CTkEntry(app, width=50)
    speed_entry.pack()

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop()



C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: {'contact', 'money', 'promo', 'cvv', 'credit', 'message', 'password', 'lottery', 'information', 'number', 'special', 'card', 'forward', 'rewards', 'aadhaar', 'chance', 'win', 'prizes', 'send', 'account', 'OTP', 'username', 'access', 'email', 'phone', 'last', 'bonus', 'lucky', 'give', 'code'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Himesh Punj\.conda\envs\env1\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 553, in _clicked
    self._command()
  File "C:\Users\Himesh Punj\AppData\Local\Temp\ipykernel_10888\707102293.py", line 157, in get_path
    probability = calculate_accuracy(path, pitch, speed)
  File "C:\Users\Himesh Punj\AppData\Local\Temp\ipykernel_10888\707102293.py", line 137, in calculate_accuracy
    text = get_large_audio_transcription(path, pitch, speed)
  File "C:\Users\Himesh Punj\AppData\Local\Temp\ipykernel_10888\707102293.py", line 58, in get_large_audio_transcription
    sound = AudioSegment.from_wav(path)
  File "C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\pydub\audio_segment.py", line 808, in from_wav
    return cls.from_file(file, 'wav', parameters=parameter

audio-chunks\chunk1.wav : Thank you for calling online support. 
audio-chunks\chunk2.wav : This is adam watching how many help you please lower down the computers volume master volume is of you make the volume zero. 
Speech recognition failed: 
audio-chunks\chunk4.wav : Make it completely zero. 
audio-chunks\chunk5.wav : Www.net. 
audio-chunks\chunk6.wav : Remote control on your computer. 
audio-chunks\chunk7.wav : I can see that you have a trojan downloaded in your computer as we believe in secure payment methodology so you have to make the payment through the check so could you please grab a cheque book handy video. 
Speech recognition failed: 
Speech recognition failed: 
audio-chunks\chunk10.wav : Ok for that my billing manager will take care ok so do you have any question before i transfer the call to my billing manager. 
audio-chunks\chunk11.wav : That's my billing manager will check ok my billing manager are the authorised person. 
audio-chunks\chunk12.wav : Hello anand how are y

In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tkinter as tk
from tkinter import filedialog
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os

# Download stopwords outside the function
nltk.download('stopwords')

# Defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

def preprocess_sentences():
    global scam_sentences, nonscam_sentences
    # Preprocess scam and non-scam sentences
    scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
    nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    max_frame_rate = 48000  # Define the maximum frame rate here (adjust as needed)

    # Pitch shifting
    if pitch != 0:
        pitch_factor = 2.0 ** (pitch / 12.0)
        new_frame_rate = int(sound.frame_rate * pitch_factor)
        if new_frame_rate > max_frame_rate:
            pitch_factor = max_frame_rate / sound.frame_rate
            new_frame_rate = max_frame_rate
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_frame_rate})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
                text = None
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text

def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]

def scam_probability(sentence):
    filtered_sentence = filter_common_words(sentence.split())
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability

def batch_scam_probability(sentences):
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)

# Create a dictionary to store the analyzed audio files and their probabilities
analyzed_audio_cache = {}

def calculate_accuracy(path, pitch=0, speed=1.0):
    global analyzed_audio_cache  # Make sure we use the global variable within the function
    # Check if the file is already analyzed, return the result from cache if available
    abs_path = os.path.abspath(path)
    if abs_path in analyzed_audio_cache:
        return analyzed_audio_cache[abs_path]
    
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text)
    
    # Store the result in cache for future use
    analyzed_audio_cache[abs_path] = prob
    return prob

def calculate_accuracy_from_text(text):
    text_list = text.split(" ")
    prob = scam_probability(text)
    return prob

def real_time_analysis():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something...")
        while True:
            try:
                audio = r.listen(source, phrase_time_limit=15)  # Set the phrase_time_limit to 15 seconds
                text = r.recognize_google(audio)
                print("Transcription:", text)
                probability = calculate_accuracy_from_text(text)
                print("Scam Probability: {:.2%}".format(probability))
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")
    customtkinter.set_default_color_theme("dark-blue")

    app = customtkinter.CTk()
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2", font=('Century Gothic', 20))
    entry_label.pack(pady=10)

    entry = customtkinter.CTkEntry(app)
    entry.pack(pady=5)

    pitch_label = customtkinter.CTkLabel(app, text="Pitch (in semitones):")
    pitch_label.pack(pady=5)

    pitch_entry = customtkinter.CTkEntry(app)
    pitch_entry.pack(pady=5)

    speed_label = customtkinter.CTkLabel(app, text="Speed (1.0 is normal):")
    speed_label.pack(pady=5)

    speed_entry = customtkinter.CTkEntry(app)
    speed_entry.pack(pady=5)

    def get_path():
        file_path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        
        probability = calculate_accuracy(file_path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))
    
    def browse_file():
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)
    
    # Create a button to browse for the audio file
    browse_button = customtkinter.CTkButton(app, text="Browse", command=browse_file)
    browse_button.pack(pady=5)

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a button for real-time analysis
    real_time_button = customtkinter.CTkButton(app, text="Real-Time Analysis", command=real_time_analysis)
    real_time_button.pack(pady=10)

    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop()

[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: {'card', 'password', 'code', 'special', 'lottery', 'information', 'rewards', 'message', 'chance', 'aadhaar', 'bonus', 'win', 'last', 'lucky', 'account', 'money', 'forward', 'access', 'username', 'cvv', 'prizes', 'credit', 'phone', 'contact', 'email', 'number', 'send', 'give', 'promo', 'OTP'}
Say something...
Speech recognition failed: 
Transcription: hello how are you doing and we are
Scam Probability: 0.00%
Speech recognition failed: 
Speech recognition failed: 
Transcription: call information
Scam Probability: 50.00%


In [ ]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tkinter as tk
from tkinter import filedialog
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os

# Download stopwords outside the function
nltk.download('stopwords')

# Defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

def preprocess_sentences():
    global scam_sentences, nonscam_sentences
    # Preprocess scam and non-scam sentences
    scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
    nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    max_frame_rate = 48000  # Define the maximum frame rate here (adjust as needed)

    # Pitch shifting
    if pitch != 0:
        pitch_factor = 2.0 ** (pitch / 12.0)
        new_frame_rate = int(sound.frame_rate * pitch_factor)
        if new_frame_rate > max_frame_rate:
            pitch_factor = max_frame_rate / sound.frame_rate
            new_frame_rate = max_frame_rate
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_frame_rate})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
                text = None
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text

def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]

def scam_probability(sentence):
    filtered_sentence = filter_common_words(sentence.split())
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability

def batch_scam_probability(sentences):
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)

# Create a dictionary to store the analyzed audio files and their probabilities
analyzed_audio_cache = {}

def calculate_accuracy(path, pitch=0, speed=1.0):
    global analyzed_audio_cache  # Make sure we use the global variable within the function
    # Check if the file is already analyzed, return the result from cache if available
    abs_path = os.path.abspath(path)
    if abs_path in analyzed_audio_cache:
        return analyzed_audio_cache[abs_path]
    
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text)
    
    # Store the result in cache for future use
    analyzed_audio_cache[abs_path] = prob
    return prob

def calculate_accuracy_from_text(text):
    text_list = text.split(" ")
    prob = scam_probability(text)
    return prob

top_real_time_analysis = False

def real_time_analysis():
    global stop_real_time_analysis
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something...")
        stop_real_time_analysis = False
        while not stop_real_time_analysis:
            try:
                audio = r.listen(source, phrase_time_limit=15)  # Set the phrase_time_limit to 15 seconds
                text = r.recognize_google(audio)
                print("Transcription:", text)
                probability = calculate_accuracy_from_text(text)
                print("Scam Probability: {:.2%}".format(probability))
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))

def stop_analysis():
    global stop_real_time_analysis
    stop_real_time_analysis = True

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")
    customtkinter.set_default_color_theme("dark-blue")

    app = customtkinter.CTk()
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2", font=('Century Gothic', 20))
    entry_label.pack(pady=10)

    entry = customtkinter.CTkEntry(app)
    entry.pack(pady=5)

    pitch_label = customtkinter.CTkLabel(app, text="Pitch (in semitones):")
    pitch_label.pack(pady=5)

    pitch_entry = customtkinter.CTkEntry(app)
    pitch_entry.pack(pady=5)

    speed_label = customtkinter.CTkLabel(app, text="Speed (1.0 is normal):")
    speed_label.pack(pady=5)

    speed_entry = customtkinter.CTkEntry(app)
    speed_entry.pack(pady=5)

    def get_path():
        file_path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        
        probability = calculate_accuracy(file_path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))
    
    def browse_file():
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)
    
    # Create a button to browse for the audio file
    browse_button = customtkinter.CTkButton(app, text="Browse", command=browse_file)
    browse_button.pack(pady=5)

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a button for real-time analysis
    real_time_button = customtkinter.CTkButton(app, text="Real-Time Analysis", command=real_time_analysis)
    real_time_button.pack(pady=10)
    # Create a button to stop the real-time analysis
    stop_button = customtkinter.CTkButton(app, text="Stop", command=stop_analysis)
    stop_button.pack(pady=10)

    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop()

C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: {'username', 'OTP', 'money', 'rewards', 'bonus', 'account', 'access', 'aadhaar', 'promo', 'number', 'cvv', 'credit', 'give', 'phone', 'lottery', 'forward', 'information', 'last', 'password', 'special', 'chance', 'contact', 'email', 'code', 'prizes', 'lucky', 'send', 'win', 'message', 'card'}
Say something...
Speech recognition failed: 
Transcription: how are you doing working on this project and we need some help regarding this can you please help us one Terry and we need some money
Scam Probability: 8.33%
Speech recognition failed: 
Speech recognition failed: 
Speech recognition failed: 
Speech recognition failed: 
Speech recognition failed: 


In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tkinter as tk
from tkinter import filedialog
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os
import threading

# Download stopwords outside the function
nltk.download('stopwords')

# Defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

def preprocess_sentences():
    global scam_sentences, nonscam_sentences
    # Preprocess scam and non-scam sentences
    scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
    nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    max_frame_rate = 48000  # Define the maximum frame rate here (adjust as needed)

    # Pitch shifting
    if pitch != 0:
        pitch_factor = 2.0 ** (pitch / 12.0)
        new_frame_rate = int(sound.frame_rate * pitch_factor)
        if new_frame_rate > max_frame_rate:
            pitch_factor = max_frame_rate / sound.frame_rate
            new_frame_rate = max_frame_rate
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_frame_rate})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
                text = None
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text

def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]

def scam_probability(sentence):
    filtered_sentence = filter_common_words(sentence.split())
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability

def batch_scam_probability(sentences):
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)

# Create a dictionary to store the analyzed audio files and their probabilities
analyzed_audio_cache = {}

def calculate_accuracy(path, pitch=0, speed=1.0):
    global analyzed_audio_cache  # Make sure we use the global variable within the function
    # Check if the file is already analyzed, return the result from cache if available
    abs_path = os.path.abspath(path)
    if abs_path in analyzed_audio_cache:
        return analyzed_audio_cache[abs_path]
    
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text)
    
    # Store the result in cache for future use
    analyzed_audio_cache[abs_path] = prob
    return prob

def calculate_accuracy_from_text(text):
    text_list = text.split(" ")
    prob = scam_probability(text)
    return prob

stop_real_time_analysis = False
stop_event = threading.Event()

def real_time_analysis():
    global stop_real_time_analysis
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something...")
        while not stop_event.is_set():
            try:
                audio = r.listen(source, phrase_time_limit=15)  # Set the phrase_time_limit to 15 seconds
                text = r.recognize_google(audio)
                print("Transcription:", text)
                probability = calculate_accuracy_from_text(text)
                print("Scam Probability: {:.2%}".format(probability))
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
            if stop_real_time_analysis:
                break

def stop_analysis():
    global stop_real_time_analysis
    stop_real_time_analysis = True
    stop_event.set()

def start_real_time_analysis():
    global stop_real_time_analysis, stop_event
    stop_real_time_analysis = False
    stop_event.clear()
    thread = threading.Thread(target=real_time_analysis)
    thread.start()

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")
    customtkinter.set_default_color_theme("dark-blue")

    app = customtkinter.CTk()
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2", font=('Century Gothic', 20))
    entry_label.pack(pady=10)

    entry = customtkinter.CTkEntry(app)
    entry.pack(pady=5)

    pitch_label = customtkinter.CTkLabel(app, text="Pitch (in semitones):")
    pitch_label.pack(pady=5)

    pitch_entry = customtkinter.CTkEntry(app)
    pitch_entry.pack(pady=5)

    speed_label = customtkinter.CTkLabel(app, text="Speed (1.0 is normal):")
    speed_label.pack(pady=5)

    speed_entry = customtkinter.CTkEntry(app)
    speed_entry.pack(pady=5)

    def get_path():
        file_path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        
        probability = calculate_accuracy(file_path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))
    
    def browse_file():
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)
    
    # Create a button to browse for the audio file
    browse_button = customtkinter.CTkButton(app, text="Browse", command=browse_file)
    browse_button.pack(pady=5)

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a button to start the real-time analysis
    start_button = customtkinter.CTkButton(app, text="Start Real-Time Analysis", command=start_real_time_analysis)
    start_button.pack(pady=10)

    # Create a button to stop the real-time analysis
    stop_button = customtkinter.CTkButton(app, text="Stop", command=stop_analysis)
    stop_button.pack(pady=10)    
    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop()

C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: {'username', 'OTP', 'phone', 'access', 'message', 'bonus', 'promo', 'send', 'lucky', 'money', 'account', 'number', 'win', 'information', 'chance', 'aadhaar', 'code', 'cvv', 'card', 'email', 'password', 'credit', 'last', 'contact', 'prizes', 'rewards', 'lottery', 'forward', 'special', 'give'}
Say something...
Transcription: hello how are you this is a call to inform that you can't has been blocked and if you need to unlock your card you need to add your card details and help us with the CVV
Scam Probability: 90.00%
Speech recognition failed: 
Speech recognition failed: 
audio-chunks\chunk3.wav : Aadarsh chief justice. 
audio-chunks\chunk4.wav : We are in the process of proceed against legal. 
Speech recognition failed: 
audio-chunks\chunk6.wav : What is the some american. 
audio-chunks\chunk7.wav : Gta game of your personal check in information along with data. 
audio-chunks\chunk8.wav : Collection agency. 
Speech recognition failed: 
audio-chunks\chunk10.wav : Ha

In [9]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tkinter as tk
from tkinter import filedialog
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os
import threading

# Download stopwords outside the function
nltk.download('stopwords')

# Defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

def preprocess_sentences():
    global scam_sentences, nonscam_sentences
    # Preprocess scam and non-scam sentences
    scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
    nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    max_frame_rate = 48000  # Define the maximum frame rate here (adjust as needed)

    # Pitch shifting
    if pitch != 0:
        pitch_factor = 2.0 ** (pitch / 12.0)
        new_frame_rate = int(sound.frame_rate * pitch_factor)
        if new_frame_rate > max_frame_rate:
            pitch_factor = max_frame_rate / sound.frame_rate
            new_frame_rate = max_frame_rate
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_frame_rate})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
                text = None
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text

def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]

def scam_probability(sentence):
    filtered_sentence = filter_common_words(sentence.split())
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability

def batch_scam_probability(sentences):
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)

# Create a dictionary to store the analyzed audio files and their probabilities
analyzed_audio_cache = {}

def calculate_accuracy(path, pitch=0, speed=1.0):
    global analyzed_audio_cache  # Make sure we use the global variable within the function
    # Check if the file is already analyzed, return the result from cache if available
    abs_path = os.path.abspath(path)
    if abs_path in analyzed_audio_cache:
        return analyzed_audio_cache[abs_path]
    
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text)
    
    # Store the result in cache for future use
    analyzed_audio_cache[abs_path] = prob
    return prob

def calculate_accuracy_from_text(text):
    text_list = text.split(" ")
    prob = scam_probability(text)
    return prob

stop_real_time_analysis = False
stop_event = threading.Event()

def real_time_analysis():
    global stop_real_time_analysis
    r = sr.Recognizer()
    while not stop_event.is_set():
        try:
            with sr.Microphone() as source:
                print("Say something...")
                audio = r.listen(source, phrase_time_limit=15)  # Set the phrase_time_limit to 15 seconds
                text = r.recognize_google(audio)
                print("Transcription:", text)
                probability = calculate_accuracy_from_text(text)
                print("Scam Probability: {:.2%}".format(probability))
                # Update the GUI label with the score
                result_label.configure(text="Scam Probability: {:.2%}".format(probability))
        except sr.UnknownValueError as e:
            print("Speech recognition failed:", str(e))
        if stop_real_time_analysis:
            break

    # Calculate the final result after real-time analysis is stopped
    final_result = calculate_final_result()
    result_label.configure(text=final_result)

def calculate_final_result():
    # Get the latest probability score from the cache (or set to 0 if not available)
    probability = analyzed_audio_cache.get(os.path.abspath(entry.get()), 0)
    return "Final Scam Probability: {:.2%}".format(probability)

def stop_analysis():
    global stop_real_time_analysis
    stop_real_time_analysis = True
    stop_event.set()

    # Calculate the final result and update the GUI label
    final_result = calculate_final_result()
    result_label.configure(text=final_result)

def start_real_time_analysis():
    global stop_real_time_analysis, stop_event
    stop_real_time_analysis = False
    stop_event.clear()
    thread = threading.Thread(target=real_time_analysis)
    thread.start()
    # Schedule the score update on the GUI every 500 milliseconds (adjust as needed)
    app.after(500, update_score)

def update_score():
    # Get the latest probability score from the cache (or set to 0 if not available)
    probability = analyzed_audio_cache.get(os.path.abspath(entry.get()), 0)
    result_label.configure(text="Scam Probability: {:.2%}".format(probability))
    if not stop_event.is_set():
        # Schedule the next score update
        app.after(10, update_score)
if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")
    customtkinter.set_default_color_theme("dark-blue")

    app = customtkinter.CTk()
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2", font=('Century Gothic', 20))
    entry_label.pack(pady=10)

    entry = customtkinter.CTkEntry(app)
    entry.pack(pady=5)

    pitch_label = customtkinter.CTkLabel(app, text="Pitch (in semitones):")
    pitch_label.pack(pady=5)

    pitch_entry = customtkinter.CTkEntry(app)
    pitch_entry.pack(pady=5)

    speed_label = customtkinter.CTkLabel(app, text="Speed (1.0 is normal):")
    speed_label.pack(pady=5)

    speed_entry = customtkinter.CTkEntry(app)
    speed_entry.pack(pady=5)

    def get_path():
        file_path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        
        probability = calculate_accuracy(file_path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))
    
    def browse_file():
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)
    
    # Create a button to browse for the audio file
    browse_button = customtkinter.CTkButton(app, text="Browse", command=browse_file)
    browse_button.pack(pady=5)

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a button to start the real-time analysis
    start_button = customtkinter.CTkButton(app, text="Start Real-Time Analysis", command=start_real_time_analysis)
    start_button.pack(pady=10)

    # Create a button to stop the real-time analysis
    stop_button = customtkinter.CTkButton(app, text="Stop", command=stop_analysis)
    stop_button.pack(pady=10)    
    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)
    
    

    app.mainloop()

[nltk_data] Downloading package stopwords to C:\Users\Himesh
[nltk_data]     Punj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Scam words considered: {'money', 'lucky', 'username', 'promo', 'OTP', 'email', 'message', 'number', 'code', 'password', 'credit', 'bonus', 'lottery', 'phone', 'information', 'contact', 'forward', 'special', 'card', 'give', 'account', 'last', 'prizes', 'cvv', 'rewards', 'send', 'win', 'access', 'aadhaar', 'chance'}
Say something...
Transcription: he is to Samuel I am happy happy unlocking could you please provide me the 12 visit card number along with the CVV at the back
Scam Probability: 96.67%
audio-chunks\chunk1.wav : Main apne 2007 model. 
audio-chunks\chunk2.wav : Aisa kahan per ho beta kahan per rahana aapka. 
audio-chunks\chunk3.wav : Main apne idhar brother mumbai international airport se baat kar raha hun. 
audio-chunks\chunk4.wav : International airport. 
audio-chunks\chunk5.wav : International airport par mera aadami chahie satta duty hai defence loan. 
Speech recognition failed: 
Speech recognition failed: 
Speech recognition failed: 
audio-chunks\chunk9.wav : Ok gadi kiske 

In [1]:
from pydub import AudioSegment
from pydub.silence import split_on_silence
import tkinter as tk
from tkinter import filedialog
import customtkinter
import speech_recognition as sr
import math
import random
import nltk
from nltk.corpus import stopwords
import os
import threading

# Download stopwords outside the function
nltk.download('stopwords')

# Defining suspicious scam words
scam_words = set("aadhaar cvv username password access information special lucky money OTP credit card bonus code chance promo win last contact prizes rewards phone send give forward email message account number lottery".split())
print("Scam words considered:", scam_words)

# Scam Sentences
scam_sentences = [
    ["I am speaking from XYZ bank. Can you send me your OTP which you received on your phone.", "True"],
    ["Please provide me with your Aadhaar Card details.", "True"],
    ["Can you tell us your Credit Card CVV number.", "True"],
    ["Please provide me with your Credit Card number.", "True"],
    ["What is your address.", "True"],
    ["You have won a lottery of 10 lakh rupees! To receive the amount, you will have to provide us with your bank account information.", "True"],
    ["I am your father's friend, he told me to ask you for some money.", "True"],
    ["Kindly tell me your Username and Password of your Google account.", "True"],
    ["We are from XYZ foundation. Donate money to bring a smile on underprivileged kids.", "True"],
    ["I am from technical support team. Can you give me access to your PC so that I can help you remove the virus.", "True"]
]

# Non-Scam Sentences
nonscam_sentences = [
    ["I am speaking from XYZ bank. We would like to verify your details.", "False"],
    ["Kindly go to our official website and provide us with the necessary details.", "False"],
    ["Visit the nearest branch of XYZ bank to process your request.", "False"],
    ["We require some additional information about you to verify your identity. Please visit the center with your KYC information.", "False"],
    ["Hey what's up! Long time no see. Let's catch up soon.", "False"],
    ["Tell your mom that XYZ aunty had called.", "False"],
    ["I am speaking from XYZ restaurant. Your parcel will arrive in few minutes.", "False"],
    ["I wanted to borrow XYZ book from you. Can you lend it to me for a few days?", "False"],
    ["This is the librarian of XYZ college. You have a pending fine, kindly pay it at the earliest.", "False"],
    ["Your monthly subscription for XYZ newspaper is due. Kindly pay it to the delivery man via Cash or Google Pay or Paytm", "False"]
]

def preprocess_sentences():
    global scam_sentences, nonscam_sentences
    # Preprocess scam and non-scam sentences
    scam_sentences = [[sentence.lower(), "True"] for sentence, _ in scam_sentences]
    nonscam_sentences = [[sentence.lower(), "False"] for sentence, _ in nonscam_sentences]

def get_large_audio_transcription(path, pitch=0, speed=1.0):
    r = sr.Recognizer()
    sound = AudioSegment.from_wav(path)

    max_frame_rate = 48000  # Define the maximum frame rate here (adjust as needed)

    # Pitch shifting
    if pitch != 0:
        pitch_factor = 2.0 ** (pitch / 12.0)
        new_frame_rate = int(sound.frame_rate * pitch_factor)
        if new_frame_rate > max_frame_rate:
            pitch_factor = max_frame_rate / sound.frame_rate
            new_frame_rate = max_frame_rate
        sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_frame_rate})

    # Speed adjustment
    if speed != 1.0:
        sound = sound.speedup(playback_speed=speed)

    # Split audio sound where silence is 700 milliseconds or more and get chunks
    chunks = split_on_silence(sound,
                              min_silence_len=500,
                              silence_thresh=sound.dBFS - 14,
                              keep_silence=500)

    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)

    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Speech recognition failed:", str(e))
                text = None
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text

    return whole_text

def filter_common_words(text_list):
    common_words = set(stopwords.words("english"))
    return [word for word in text_list if word not in common_words]

def scam_probability(sentence):
    filtered_sentence = filter_common_words(sentence.split())
    if set(filtered_sentence).isdisjoint(scam_words):
        return 0
    else:
        scam_indices = [i for i, word in enumerate(filtered_sentence) if word in scam_words]
        index_diff = [scam_indices[i] - scam_indices[i - 1] for i in range(1, len(scam_indices))]
        if len(index_diff) == 0:
            probability = 1 / len(filtered_sentence)
        elif len(scam_indices) / len(filtered_sentence) > 0.5:
            return len(scam_indices) / len(filtered_sentence)
        else:
            probability = 1 - (sum(index_diff) / len(index_diff)) / (2 * len(filtered_sentence))
    return probability

def batch_scam_probability(sentences):
    probabilities = [scam_probability(sentence) for sentence, _ in sentences if scam_probability(sentence) != 0]
    print("\nThe individual probabilities are:", [round(i * 100, 2) for i in probabilities])
    if len(probabilities) == 0:
        return 0
    return math.pow((sum([x ** 8 for x in probabilities]) / len(probabilities)), 0.125)

# Create a dictionary to store the analyzed audio files and their probabilities
analyzed_audio_cache = {}

def calculate_accuracy(path, pitch=0, speed=1.0):
    # Check if the file is already analyzed, return the result from cache if available
    abs_path = os.path.abspath(path)
    if abs_path in analyzed_audio_cache:
        return analyzed_audio_cache[abs_path]
    
    text = get_large_audio_transcription(path, pitch, speed)
    text_list = text.split(" ")
    prob = scam_probability(text)
    
    # Store the result in cache for future use
    analyzed_audio_cache[abs_path] = prob
    return prob

def calculate_accuracy_from_text(text):
    text_list = text.split(" ")
    prob = scam_probability(text)
    return prob

stop_real_time_analysis = False
stop_event = threading.Event()

def real_time_analysis():
    r = sr.Recognizer()
    while not stop_event.is_set():
        try:
            with sr.Microphone() as source:
                print("Say something...")
                audio = r.listen(source, phrase_time_limit=15)  # Set the phrase_time_limit to 15 seconds
                text = r.recognize_google(audio)
                print("Transcription:", text)
                probability = calculate_accuracy_from_text(text)
                print("Scam Probability: {:.2%}".format(probability))
                # Update the GUI label with the score
                result_label.configure(text="Scam Probability: {:.2%}".format(probability))
        except sr.UnknownValueError as e:
            print("Speech recognition failed:", str(e))
        if stop_real_time_analysis:
            break

def calculate_final_result():
    # Get the latest probability score from the cache (or set to 0 if not available)
    probability = analyzed_audio_cache.get(os.path.abspath(entry.get()), 0)
    return "Final Scam Probability: {:.2%}".format(probability)

def stop_analysis():
    global stop_real_time_analysis
    stop_real_time_analysis = True
    stop_event.set()

    # Calculate the final result after real-time analysis is stopped
    final_result = calculate_final_result()
    result_label.configure(text=final_result)

    # Reset the label after a brief delay to show the final result
    app.after(2000, reset_label)

def reset_label():
    result_label.configure(text="")

def start_real_time_analysis():
    global stop_real_time_analysis, stop_event
    stop_real_time_analysis = False
    stop_event.clear()
    thread = threading.Thread(target=real_time_analysis)
    thread.start()

if __name__ == '__main__':
    customtkinter.set_appearance_mode("light")
    customtkinter.set_default_color_theme("dark-blue")

    app = customtkinter.CTk()
    app.geometry("300x500")
    app.title("Scam Call Detector v1.2")

    entry_label = customtkinter.CTkLabel(app, text="Scam detector v1.2", font=('Century Gothic', 20))
    entry_label.pack(pady=10)

    entry = customtkinter.CTkEntry(app)
    entry.pack(pady=5)

    pitch_label = customtkinter.CTkLabel(app, text="Pitch (in semitones):")
    pitch_label.pack(pady=5)

    pitch_entry = customtkinter.CTkEntry(app)
    pitch_entry.pack(pady=5)

    speed_label = customtkinter.CTkLabel(app, text="Speed (1.0 is normal):")
    speed_label.pack(pady=5)

    speed_entry = customtkinter.CTkEntry(app)
    speed_entry.pack(pady=5)

    def get_path():
        file_path = entry.get()
        pitch = int(pitch_entry.get())
        speed = float(speed_entry.get())
        
        probability = calculate_accuracy(file_path, pitch, speed)

        # Update the label text using the configure method
        result_label.configure(text="Scam Probability: {:.2%}".format(probability))
    
    def browse_file():
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if file_path:
            entry.delete(0, tk.END)
            entry.insert(0, file_path)
    
    # Create a button to browse for the audio file
    browse_button = customtkinter.CTkButton(app, text="Browse", command=browse_file)
    browse_button.pack(pady=5)

    # Create a button to trigger the analysis
    analyze_button = customtkinter.CTkButton(app, text="Analyze", command=get_path)
    analyze_button.pack(pady=10)

    # Create a button to start the real-time analysis
    start_button = customtkinter.CTkButton(app, text="Start Real-Time Analysis", command=start_real_time_analysis)
    start_button.pack(pady=10)

    # Create a button to stop the real-time analysis
    stop_button = customtkinter.CTkButton(app, text="Stop", command=stop_analysis)
    stop_button.pack(pady=10)
    
    # Create a label to display the result
    result_label = customtkinter.CTkLabel(app, text="")
    result_label.pack(pady=10)

    app.mainloop()


C:\Users\Himesh Punj\.conda\envs\env1\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


Scam words considered: {'give', 'information', 'win', 'bonus', 'send', 'contact', 'OTP', 'phone', 'promo', 'prizes', 'chance', 'forward', 'message', 'special', 'number', 'code', 'money', 'email', 'credit', 'password', 'card', 'account', 'rewards', 'aadhaar', 'last', 'lottery', 'access', 'lucky', 'username', 'cvv'}
Say something...
Transcription: hi this is Samuel I want to do you find your amount into your bank account could you please provide me 12 digit card number along with the CVV so that can initiate the refund
Scam Probability: 90.79%
Speech recognition failed: 
Speech recognition failed: 
audio-chunks\chunk3.wav : Aadarsh chief justice. 
audio-chunks\chunk4.wav : We are in the process of proceed against legal. 
Speech recognition failed: 
audio-chunks\chunk6.wav : What is the some american. 
audio-chunks\chunk7.wav : Gta game of your personal check in information along with data. 
audio-chunks\chunk8.wav : Collection agency. 
Speech recognition failed: 
audio-chunks\chunk10.wav